In [1]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import xmltodict
import pandas as pd
import time
import os  

In [2]:
#open up folder, probably get all game csv to append to
folder = r"C:\Users\peter\Desktop\tdi_capstone"

In [3]:
master_df = pd.read_csv(folder + "\\bgg_game_info.csv", index_col = 0)
master_df = master_df.reset_index(drop=True)
master_df.tail()

,bgg_id,name,bgg_type,thumbnail,image,description,year_published,players_min,players_max,playingtime,...,boardgameartist,boardgamecategory,boardgamedesigner,boardgamefamily,boardgamemechanic,boardgamepublisher,boardgameimplementation,boardgameexpansion,boardgamecompilation,boardgameintegration
833,962,Contango,boardgame,https://cf.geekdo-images.com/hjO7d4ngm0rV_vfI0...,https://cf.geekdo-images.com/hjO7d4ngm0rV_vfI0...,CONTANGO &ldquo;The Ultimate Stock Market Game...,2000,2,6,90,...,NaN,"['Abstract Strategy', 'Deduction', 'Dice', 'Ec...",['S. Schwab'],NaN,"['Auction/Bidding', 'Commodity Speculation', '...",['Contango Concepts'],NaN,NaN,NaN,NaN
834,963,The Gardens of the Alhambra,boardgame,https://cf.geekdo-images.com/MtvT2-mmZilBcegqn...,https://cf.geekdo-images.com/MtvT2-mmZilBcegqn...,The Gardens of Alhambra is an abstract influen...,1993,2,4,60,...,"['Christian Bohr', 'Jo Hartwig']",['Abstract Strategy'],['Dirk Henn'],"['Cities: Granada (Andalusia, Spain)', 'Game: ...","['Area Majority / Influence', 'Tile Placement']","['Queen Games', 'db-Spiele']",NaN,NaN,['Alhambra: Big Box Special Edition'],NaN
835,964,Kapitän Wackelpudding,boardgame,https://cf.geekdo-images.com/zf6Py2NWtH17gSZAo...,https://cf.geekdo-images.com/zf6Py2NWtH17gSZAo...,"Wackel is German for wobbly, and Captain Jelly...",1994,2,5,30,...,NaN,"['Action / Dexterity', ""Children's Game"", 'Nau...","['Sean McGuire', 'Thomas McGuire']",NaN,"['Pick-up and Deliver', 'Point to Point Moveme...","['Peri Spiele', 'Perner Produktions GmbH']",NaN,NaN,NaN,NaN
836,965,Ben Hurt,boardgame,https://cf.geekdo-images.com/cEFhesBe4jlTlsojS...,https://cf.geekdo-images.com/cEFhesBe4jlTlsojS...,A chariot-racing game with pre-race rounds of ...,1996,4,8,60,...,['James Ernest'],"['Ancient', 'Dice', 'Racing']",['James Ernest'],['Ancient: Rome'],"['Auction/Bidding', 'Betting and Bluffing', 'R...","['Cheapass Games', 'Truant Spiele']",NaN,NaN,NaN,NaN
837,967,Kommissar X,boardgame,https://cf.geekdo-images.com/9RfQI4SKfFeB0wbK-...,https://cf.geekdo-images.com/9RfQI4SKfFeB0wbK-...,The Police Commissioner is on the hunt for esc...,1992,2,6,30,...,NaN,"['Card Game', 'Memory']",['Nik Sewell'],NaN,NaN,['AMIGO'],NaN,NaN,NaN,NaN


In [4]:
#enter game you want 
game_id = 977

#start
start_game = game_id

#enter how many games to get?
number_games = 33

#enter starting page, probably 1
page_id = 1

In [5]:
while game_id < (start_game + number_games):

    #get url
    url = f"https://boardgamegeek.com/xmlapi2/thing?id={game_id}&stats=1.xml"

    #get xml file
    #had issue try three times to ping site
    # https://stackoverflow.com/questions/15431044/can-i-set-max-retries-for-requests-request
    
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)

    response = session.get(url)
    
    data = xmltodict.parse(response.content)
    
    #IF bgg_id exist, go on, if not. reset page, add one to game, and move to next game, continue
    #if doesn't exist, 
    if "item" not in data['items']:
        #reset page id
        page_id = 1

        #go to next game
        game_id += 1
        
        time.sleep(6) # Sleep for 6 seconds
        
        #go to next while loop
        continue
    
    #if not a boardgame type, move on as well
    if data['items']['item']['@type'] != 'boardgame':
        #reset page id
        page_id = 1

        #go to next game
        game_id += 1
        
        time.sleep(6) # Sleep for 6 seconds
        
        #go to next while loop
        continue
    
    #get game id
    bgg_id = int(data['items']['item']['@id'])
    
    #make new dataframe
    d = {'bgg_id': [bgg_id]}
    df = pd.DataFrame(data=d)
    
    
    #get name object, if it has alternate names will be a list, otherwise a dic
    name_object = data['items']['item']['name']
    
    #get name if a list, or dic
    if isinstance(name_object, list):
        df['name'] = data['items']['item']['name'][0]['@value'] #str, if alternate names then get first in list. Primary game name
    if isinstance(name_object, dict):
        df['name'] = data['items']['item']['name']['@value'] #str, game name


    #board game type, and images
    df['bgg_type'] = data['items']['item']['@type'] #str
    
    if "thumbnail" in data['items']['item']:
        df['thumbnail'] = data['items']['item']['thumbnail'] #str to url of image
    else:
        df['thumbnail'] = "NaN"
    if "image" in data['items']['item']:
        df['image'] = data['items']['item']['image'] #str to url of image
    else:
        df['image'] = "NaN"


    df['description'] = data['items']['item']['description'] #str, paragraph
    df['year_published'] = int(data['items']['item']['yearpublished']['@value']) #int

    #players, time, and age
    df['players_min'] = int(data['items']['item']['minplayers']['@value']) #int
    df['players_max'] = int(data['items']['item']['maxplayers']['@value']) #int
    df['playingtime'] = int(data['items']['item']['playingtime']['@value']) #int
    df['playingtime_min'] = int(data['items']['item']['minplaytime']['@value']) #int
    df['playingtime_max'] = int(data['items']['item']['maxplaytime']['@value']) #int
    df['age_min'] = int(data['items']['item']['minage']['@value'])

    #statistics, rating
    df['rating_number'] = int(data['items']['item']['statistics']['ratings']['usersrated']['@value']) #int
    df['rating_average']= float(data['items']['item']['statistics']['ratings']['average']['@value']) #float
    df['rating_baysaverage'] = float(data['items']['item']['statistics']['ratings']['bayesaverage']['@value']) #float
    df['rating_stddev']  = float(data['items']['item']['statistics']['ratings']['stddev']['@value']) #float
    df['rating_median'] = float(data['items']['item']['statistics']['ratings']['median']['@value']) #float, maybe all zero
    df['comments_number'] = int(data['items']['item']['statistics']['ratings']['numcomments']['@value']) #int

    #statistic ranks
    
    #get rank object, if list, then there is overall and strategy ranking, otherwise, just overall
    rank_object = (data['items']['item']['statistics']['ratings']['ranks']['rank'])
    
    if isinstance(rank_object, list):
        
        if isinstance(data['items']['item']['statistics']['ratings']['ranks']['rank'][0]['@value'], str):
            df['rank_overall'] = "NaN"
        else:
            df['rank_overall'] = int(data['items']['item']['statistics']['ratings']['ranks']['rank'][0]['@value']) #int
            
        df['rank_strategy'] = "Nan" #int(data['items']['item']['statistics']['ratings']['ranks']['rank'][0]['@value']) #int
    if isinstance(rank_object, dict):
        
        if isinstance(data['items']['item']['statistics']['ratings']['ranks']['rank']['@value'], str):
            df['rank_overall'] = "NaN"
        else:
            df['rank_overall'] = int(data['items']['item']['statistics']['ratings']['ranks']['rank']['@value']) #int
        
    #statistics complexity
    df['complexity_number'] = int(data['items']['item']['statistics']['ratings']['numweights']['@value']) #int
    df['complexity_average'] = float(data['items']['item']['statistics']['ratings']['averageweight']['@value']) #float

    #statistics marketplace
    df['owned'] = int(data['items']['item']['statistics']['ratings']['owned']['@value'])
    df['trading'] = int(data['items']['item']['statistics']['ratings']['trading']['@value'])
    df['wanting'] = int(data['items']['item']['statistics']['ratings']['wanting']['@value'])
    df['wishing'] = int(data['items']['item']['statistics']['ratings']['wishing']['@value'])

    #generate list of dics of the category data
    master_dic_list = []

    #get list of links
    dic_list = data['items']['item']['link']

    #turn into one list
    master_dic_list.extend(dic_list)

    #make dataframe
    df_links = pd.DataFrame(master_dic_list)

    #drop id column, I think corresponds to a numerical code for the categories
    df_links = df_links.drop('@id', axis=1)

    #group by value, into lists
    df_grouped = df_links.groupby('@type')['@value'].apply(list).reset_index()

    #add game column with the game ID
    df_grouped['bgg_id'] = bgg_id

    #pivot into one row
    df_grouped = df_grouped.pivot(index='bgg_id', columns='@type', values='@value')

    df_grouped = df_grouped.reset_index()
    
    #merge the df with the df_grouped, the generic info with the category info
    df_combined =  df.merge(df_grouped, on='bgg_id')  #pd.concat([df, df_grouped], axis=1, join="inner")

    
    #concat the master_df with the new combined row
    master_df = pd.concat([master_df, df_combined], ignore_index=True, sort=False) #ignore_index=True
    
    time.sleep(6) # Sleep for 6 seconds
    
    #go to next game
    game_id += 1
    
    

In [6]:
master_df

,bgg_id,name,bgg_type,thumbnail,image,description,year_published,players_min,players_max,playingtime,...,boardgameartist,boardgamecategory,boardgamedesigner,boardgamefamily,boardgamemechanic,boardgamepublisher,boardgameimplementation,boardgameexpansion,boardgamecompilation,boardgameintegration
0,1,Die Macher,boardgame,https://cf.geekdo-images.com/rpwCZAjYLD940NWwP...,https://cf.geekdo-images.com/rpwCZAjYLD940NWwP...,Die Macher is a game about seven sequential po...,1986,3,5,240,...,"['Marcus Gschwendtner', 'Harald Lieske']","['Economic', 'Negotiation', 'Political']",['Karl-Heinz Schmiel'],"['Country: Germany', 'Political: Elections', '...","['Alliances', 'Area Majority / Influence', 'Au...","['Hans im Glück', 'Moskito Spiele', 'Ediciones...",NaN,NaN,NaN,NaN
1,2,Dragonmaster,boardgame,https://cf.geekdo-images.com/oQYhaJx5Lg3KcGis2...,https://cf.geekdo-images.com/oQYhaJx5Lg3KcGis2...,Dragonmaster is a trick-taking card game based...,1981,3,4,30,...,['Bob Pepper'],"['Card Game', 'Fantasy']","['G. W. ""Jerry"" D\'Arcey']","['Components: Gems/Crystals', 'Creatures: Drag...",['Trick-taking'],"['E.S. Lowe', 'Milton Bradley']","['Indulgence', ""Coup d'État""]",NaN,NaN,NaN
2,3,Samurai,boardgame,https://cf.geekdo-images.com/o9-sNXmFS_TLAb7Zl...,https://cf.geekdo-images.com/o9-sNXmFS_TLAb7Zl...,Samurai is set in medieval Japan. Players comp...,1998,2,4,60,...,['Franz Vohwinkel'],"['Abstract Strategy', 'Medieval']",['Reiner Knizia'],['Components: Map (Continental / National scal...,"['Area Majority / Influence', 'Hand Management...","['Fantasy Flight Games', 'Hans im Glück', '999...",['Samurai: The Card Game'],NaN,NaN,NaN
3,4,Tal der Könige,boardgame,https://cf.geekdo-images.com/nYiYhUlatT2DpyXaJ...,https://cf.geekdo-images.com/nYiYhUlatT2DpyXaJ...,When you see the triangular box and the luxuri...,1992,2,4,60,...,['Thomas di Paolo'],['Ancient'],['Christian Beierer'],"['Containers: Triangular Boxes', 'Country: Egy...","['Action Points', 'Area Majority / Influence',...",['KOSMOS'],NaN,NaN,NaN,NaN
4,5,Acquire,boardgame,https://cf.geekdo-images.com/3C--kJRhi6kTPHsr9...,https://cf.geekdo-images.com/3C--kJRhi6kTPHsr9...,"In Acquire, each player strategically invests ...",1964,2,6,90,...,"['Scott Okumura', 'Peter Whitley']","['Economic', 'Territory Building']",['Sid Sackson'],['Series: 3M Bookshelf Series'],"['Hand Management', 'Investment', 'Market', 'S...","['3M', 'The Avalon Hill Game Co', 'Avalon Hill...",NaN,['1963 World Map Conversion Kit for the Game o...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
858,1004,Der Feuersalamander,boardgame,https://cf.geekdo-images.com/NC6R_g2l8LuKZNxek...,https://cf.geekdo-images.com/NC6R_g2l8LuKZNxek...,Der Feuersalamander is a deduction game with a...,1987,2,4,30,...,[Johann Rüttinger],"[Deduction, Fantasy, Memory]",[Johann Rüttinger],[Series: Noris' Fantasy trilogy (Noris Spiele)],"[Dice Rolling, Hand Management, Memory, Roll /...","[Dourios, Noris Spiele]",NaN,NaN,NaN,NaN
859,1005,Der Tiger ist los,boardgame,https://cf.geekdo-images.com/4SqnQG5MQIBzJQJvK...,https://cf.geekdo-images.com/4SqnQG5MQIBzJQJvK...,Cooperative game in which the players try to c...,1998,2,4,15,...,[Oliver Freudenreich],"[Animals, Children's Game]",[Edith Schlichting],[Animals: Tigers],"[Cooperative Game, Roll / Spin and Move]","[AMIGO, Edith Schlichting Company]",NaN,NaN,NaN,NaN
860,1006,Die Erbraffer,boardgame,https://cf.geekdo-images.com/dZS6dMrOekt7tWF3y...,https://cf.geekdo-images.com/dZS6dMrOekt7tWF3y...,Everyone dreams of inheriting a vast fortune a...,1994,2,6,90,...,[Stephanie Delfmann],NaN,"[Nik Sewell, Jeremy Shaw]",NaN,NaN,[Ravensburger],NaN,NaN,NaN,NaN
861,1007,Die Hanse,boardgame,https://cf.geekdo-images.com/fQKvNNrrnLcQZC0_f...,https://cf.geekdo-images.com/fQKvNNrrnLcQZC0_f...,"From the publisher:&#10;&#10;The players, as m...",1993,3,6,75,...,[Tom Schoeps],"[Medieval, Nautical, Negotiation]",[Tom Schoeps],[Components: Map (Continental / National scale...,"[Pick-up 

In [7]:
#save master csv
master_df.to_csv(folder + "\\bgg_game_info.csv")

In [ ]:
df_merge = pd.concat([master_df, df_combined], ignore_index=True, sort=False) #ignore_index=True
df_merge

In [ ]:
#save master csv
master_df.to_csv(folder + "\\bgg_game_info.csv")

In [ ]:
#get url
url = f"https://boardgamegeek.com/xmlapi2/thing?id={game_id}&stats=1.xml"

#get xml file
response = requests.get(url)
data = xmltodict.parse(response.content)

#get game id
bgg_id = int(data['items']['item']['@id'])

#master_dic_list = []

In [ ]:
data['items']['item']['@type']

In [ ]:
bgg_id = int(data['items']['item']['@id']) #int

In [ ]:
d = {'bgg_id': [bgg_id]}
df = pd.DataFrame(data=d)

In [ ]:
name_object = data['items']['item']['name']

if isinstance(name_object, list):
    df['name'] = data['items']['item']['name'][0]['@value'] #str, if alternate names then get first in list. Primary game name
if isinstance(name_object, dict):
    df['name'] = data['items']['item']['name']['@value'] #str, game name


#board game type, and images
df['bgg_type'] = data['items']['item']['@type'] #str
df['thumbnail'] = data['items']['item']['thumbnail'] #str to url of image
df['image'] = data['items']['item']['image'] #str to url of image

    
df['description'] = data['items']['item']['description'] #str, paragraph
df['year_published'] = int(data['items']['item']['yearpublished']['@value']) #int

#players, time, and age
df['players_min'] = int(data['items']['item']['minplayers']['@value']) #int
df['players_max'] = int(data['items']['item']['maxplayers']['@value']) #int
df['playingtime'] = int(data['items']['item']['playingtime']['@value']) #int
df['playingtime_min'] = int(data['items']['item']['minplaytime']['@value']) #int
df['playingtime_max'] = int(data['items']['item']['maxplaytime']['@value']) #int
df['age_min'] = int(data['items']['item']['minage']['@value'])

#statistics, rating
df['rating_number'] = int(data['items']['item']['statistics']['ratings']['usersrated']['@value']) #int
df['rating_average']= float(data['items']['item']['statistics']['ratings']['average']['@value']) #float
df['rating_baysaverage'] = float(data['items']['item']['statistics']['ratings']['bayesaverage']['@value']) #float
df['rating_stddev']  = float(data['items']['item']['statistics']['ratings']['stddev']['@value']) #float
df['rating_median'] = float(data['items']['item']['statistics']['ratings']['median']['@value']) #float, maybe all zero
df['comments_number'] = int(data['items']['item']['statistics']['ratings']['numcomments']['@value']) #int

#statistic ranks
df['rank_overall'] = int(data['items']['item']['statistics']['ratings']['ranks']['rank'][0]['@value']) #int
df['rank_strategy'] = int(data['items']['item']['statistics']['ratings']['ranks']['rank'][0]['@value']) #int

#statistics complexity
df['complexity_number'] = int(data['items']['item']['statistics']['ratings']['numweights']['@value']) #int
df['complexity_average'] = float(data['items']['item']['statistics']['ratings']['averageweight']['@value']) #float

#statistics marketplace
df['owned'] = int(data['items']['item']['statistics']['ratings']['owned']['@value'])
df['trading'] = int(data['items']['item']['statistics']['ratings']['trading']['@value'])
df['wanting'] = int(data['items']['item']['statistics']['ratings']['wanting']['@value'])
df['wishing'] = int(data['items']['item']['statistics']['ratings']['wishing']['@value'])





master_dic_list = []

#get list of links
dic_list = data['items']['item']['link']

#turn into one list
master_dic_list.extend(dic_list)

#make dataframe
df_links = pd.DataFrame(master_dic_list)

#drop id column, I think corresponds to a numerical code for the categories
df_links = df_links.drop('@id', axis=1)

#group by value, into lists
df_grouped = df_links.groupby('@type')['@value'].apply(list).reset_index()

#add game column with the game ID
df_grouped['bgg_id'] = bgg_id

#pivot into one row
df_grouped = df_grouped.pivot(index='bgg_id', columns='@type', values='@value')

df_grouped = df_grouped.reset_index()

df_combined = pd.concat([df, df_grouped], axis=1, join="inner")

df_combined


In [ ]:
#poll, 

In [ ]:
#combine with all the others,

In [ ]:
df_combined1 = df_combined

In [ ]:
df_combined2 = df_combined

In [ ]:
df_combined1

In [ ]:
df_merge = pd.concat([df_combined1, df_combined2], ignore_index=True, sort=False)
df_merge



In [ ]:
folder

In [ ]:
master_df = df_combined

In [ ]:
#save master csv
master_df.to_csv(folder + "\\bgg_game_info.csv")

In [ ]:
#list of dics

In [ ]:
while game_id < (start_game + number_games):

    #get url
    url = f"https://boardgamegeek.com/xmlapi2/thing?id={game_id}&ratingcomments=1&page={page_id}&pagesize=100.xml"

    #get xml file
    response = requests.get(url)
    data = xmltodict.parse(response.content)

    #get game id
    bgg_id = int(data['items']['item']['@id'])

    master_dic_list = []

    #if there are ratings, get them, if not, stop
    while 'comment' in data['items']['item']['comments'].keys():

        #get list of comments
        dic_list = data['items']['item']['comments']['comment']

        #from xml, get list of dictionaries which contain user/ratings/comment
        dic_list = data['items']['item']['comments']['comment']

        master_dic_list.extend(dic_list)

        #then add 1 to page, ,pause 5 seconds, then get new page
        page_id += 1
        time.sleep(5) # Sleep for 5 seconds

        #get url
        url = f"https://boardgamegeek.com/xmlapi2/thing?id={game_id}&ratingcomments=1&page={page_id}&pagesize=100.xml"

        #get xml file
        response = requests.get(url)
        data = xmltodict.parse(response.content)


    #make df from dict_list
    df = pd.DataFrame(master_dic_list)

    #rename columns
    df = df.rename(columns={"@username": "Username", "@rating": "Rating", "@value" : "Comment"})

    #convert rating from string to float
    df['Rating'] = df['Rating'].astype(float)

    #add the current game id to the dataframe
    df['Bggid'] = bgg_id

    #write game as individual file
    df.to_csv(folder + "\\" + str(bgg_id) + ".csv")

    #concat master dataframe with new dataframe
    new_master = pd.concat([master_df, df])

    #reset page id
    page_id = 1

    #go to next game
    game_id += 1